This is a zero-shot model attempt using pranaydeeps/Ancient-Greek-BERT. The model was trained on ancient Greek text, but it is not capable of performing zero-shot classification and thus fails to be performant.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import csv

In [32]:
tokenizer = AutoTokenizer.from_pretrained("pranaydeeps/Ancient-Greek-BERT")
model = AutoModel.from_pretrained("pranaydeeps/Ancient-Greek-BERT")  

classifier = pipeline(
    "zero-shot-classification", model=model, tokenizer=tokenizer, device=0
)

Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [33]:
df = pd.read_csv("../Lemmatizer-GRK/greek_dictionary/nouns.csv", encoding="utf8", sep="\t")

print(df.head())
print(df.columns)


   Unnamed: 0        FPP                        English  Sentiment
0           0  αβδηριτης                a man of Abdera        NaN
1           1  αβελτερια  silliness, stupidity, fatuity        NaN
2           2      αβιοι     without a living, starving        NaN
3           3    αβλαβια              freedom from harm        NaN
4           4      αβλης                     not thrown        NaN
Index(['Unnamed: 0', 'FPP', 'English', 'Sentiment'], dtype='object')


In [34]:
text_labels = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",
    "trust",
]
sample_text = "ἐγώ ὑμᾶς ἐπαινῶ"

In [35]:
# run the model against the sample text
# model score outputs indicate a confidence rating that the model has applied to the sample text
classifier(sample_text, text_labels, multi_label=False)

KeyError: 'logits'

In [15]:
# run prediction on entire given corpus.
# text_column is the column of the dataframe that holds the text in question
# text_labels is the emotional labels that should be addressed


def predict_sentiment(df, text_column, text_labels):
    result_list = []
    for index, row in df.iterrows():
        sequence_to_classify = row[text_column]
        result = classifier(sequence_to_classify, text_labels, multi_label=False)
        result["sentiment"] = result["labels"][0]
        result["score"] = result["scores"][0]
        result_list.append(result)
    result_df = pd.DataFrame(result_list)[["sequence", "sentiment", "score"]]
    result_df = pd.merge(df, result_df, left_on="text", right_on="sequence", how="left")
    return result_df

def predict_sentiment_batch(df, text_column, text_labels, batch_size):
    texts = df[text_column].tolist()
    result_list = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        results = classifier(batch, text_labels, multi_label=False)
        
        for text, r in zip(batch, results):
            result_list.append({
                "sequence": text,
                "sentiment": r["labels"][0],
                "score": r["scores"][0],
            })
    
    result_df = pd.DataFrame(result_list)
    return df.merge(result_df, left_on=text_column, right_on="sequence", how="left")

In [16]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.data.dataloader")

results_df = predict_sentiment_batch(df, "FPP", text_labels, 16)

In [17]:
print(results_df)

       Unnamed: 0        FPP                         English  Sentiment  \
0               0  αβδηριτης                 a man of Abdera        NaN   
1               1  αβελτερια   silliness, stupidity, fatuity        NaN   
2               2      αβιοι      without a living, starving        NaN   
3               3    αβλαβια               freedom from harm        NaN   
4               4      αβλης                      not thrown        NaN   
...           ...        ...                             ...        ...   
13003       12585     ωφελιη  help, aid, succour, assistance        NaN   
13004       12586     ωφελημ                        a useful        NaN   
13005       12587   ωφελησις              a helping, aiding;        NaN   
13006       12588      ωχρος               paleness, wanness        NaN   
13007       12589    ωχροτης                        paleness        NaN   

        sequence     sentiment     score  
0      αβδηριτης         anger  0.137011  
1      αβελτε

In [19]:
results_df.to_csv("./greekBert_sentiment")